# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [43]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [44]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
A tailored analysis of data from OMARS designs	Talk	talk-1	European Network for Business and Industrial Statistics (ENBIS)	2021-09-14	Online	https://conferences.enbis.org/event/11/contributions/128/	
One D-optimal main effects design is not the other	Talk	talk-2	European Network for Business and Industrial Statistics (ENBIS)	2022-06-27	"Trondheim, Norway"	https://conferences.enbis.org/event/18/contributions/259/	
D- and A- optimal screening designs. What if n is not a multiple of 4?	Talk	talk-3	International Conference on Design of Experiments (ICODOE)	2023-05-11	"Memphis, TN, USA"	https://www.memphis.edu/msci/icodoe22/pdfs/icodoe-2022-booklet.pdf	
Multi-criteria evaluation and selection of experimental designs from a catalog	Talk	talk-4	Fall Technical Conference (FTC)	2023-10-06	"Raleigh, NC, USA"	https://falltechnicalconference.org/2023-abstracts/#6A1	
Optimal experimental design when not all tests are equally expensive	Tal

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [45]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,A tailored analysis of data from OMARS designs,Talk,talk-1,European Network for Business and Industrial S...,2021-09-14,Online,https://conferences.enbis.org/event/11/contrib...,NaN
1,One D-optimal main effects design is not the o...,Talk,talk-2,European Network for Business and Industrial S...,2022-06-27,"Trondheim, Norway",https://conferences.enbis.org/event/18/contrib...,NaN
2,D- and A- optimal screening designs. What if n...,Talk,talk-3,International Conference on Design of Experime...,2023-05-11,"Memphis, TN, USA",https://www.memphis.edu/msci/icodoe22/pdfs/ico...,NaN
3,Multi-criteria evaluation and selection of exp...,Talk,talk-4,Fall Technical Conference (FTC),2023-10-06,"Raleigh, NC, USA",https://falltechnicalconference.org/2023-abstr...,NaN
4,Optimal experimental design when not all tests...,Talk,talk-5,European Network for Business and Industrial S...,2024-09-17,"Leuven, Belgium",https://conferences.enbis.org/event/34/contrib...,NaN
5,Minimally aliased D- and A-optimal main-effect...,Talk,talk-6,Fall Technical Conference (FTC),2024-10-09,"Nashville, TN, USA",https://falltechnicalconference.org/2024-abstr...,NaN
6,D- and A- optimal main effects designs for scr...,Seminar,sem-1,KU Leuven,2022-12-06,"Leuven, Belgium",http://www.experimental-design.eu/doe-it/,NaN
7,A study of design-based model selection methods,Seminar,sem-2,KU Leuven,2020-04-24,"Leuven, Belgium",http://www.experimental-design.eu/doe-it/,NaN
8,Finding the Optimal Cocktail,Seminar,sem-3,KU Leuven,2019-09-24,"Leuven, Belgium",http://www.experimental-design.eu/doe-it/,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [46]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [47]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [48]:
!ls ../_talks

19-sem-3.md          2022-12-06-sem-1.md  22-sem-1.md
20-sem-2.md          2023-05-11-talk-3.md 22-talk-2.md
2019-09-24-sem-3.md  2023-10-06-talk-4.md 23-talk-3.md
2020-04-24-sem-2.md  2024-09-17-talk-5.md 23-talk-4.md
2021-09-14-talk-1.md 2024-10-09-talk-6.md 24-talk-5.md
2022-06-27-talk-2.md 21-talk-1.md         24-talk-6.md


In [49]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
